In [4]:
#importamos los paquetes necesarios
import consultas as cst
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt   
import time
import math
import datetime as dt

In [ ]:
query=f'''
SELECT PERIODO_PROCESO_CODIGO,
       SUBSCRIBER_ID, 
       MSISDN,
       ESTADO,
       MOVIMIENTO_TIPO_NOMBRE,
       PLAN_COMERCIAL_DK,
       CONSULTOR_DK,
       OPERADOR_PORTA_ORIGEN,
       OPERADOR_PORTA_DESTINO
       
FROM (
SELECT A.*,
       ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY PERIODO_PROCESO_CODIGO DESC ) AS RN  
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE SERVICIO='Postpaid'
AND PERIODO_PROCESO_CODIGO >=202306
AND SUBSCRIBER_ID IN (
SELECT SUBSCRIBER_ID FROM DP_GESTION_CHURN_202306
) ) WHERE RN=1

'''

In [54]:
query=f'''
WITH BASE AS (
SELECT *
FROM DP_GESTION_CHURN_202306
),
BASE2 AS (
SELECT PERIODO_PROCESO_CODIGO,
       SUBSCRIBER_ID, 
       MSISDN,
       ESTADO,
       MOVIMIENTO_TIPO_NOMBRE,
       PLAN_COMERCIAL_DK,
       CONSULTOR_DK,
       OPERADOR_PORTA_ORIGEN,
       OPERADOR_PORTA_DESTINO
       
FROM (
SELECT A.*,
       ROW_NUMBER() OVER(PARTITION BY SUBSCRIBER_ID ORDER BY PERIODO_PROCESO_CODIGO DESC ) AS RN  
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE SERVICIO='Postpaid'
AND PERIODO_PROCESO_CODIGO >=202306
AND SUBSCRIBER_ID IN (
SELECT SUBSCRIBER_ID FROM DP_GESTION_CHURN_202306
) ) WHERE RN=1
)
SELECT *
FROM BASE A 
LEFT JOIN BASE2 B ON A.SUBSCRIBER_ID=B.SUBSCRIBER_ID
'''
base_riesgo_jun=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [55]:
base_riesgo_jun.to_csv('base_riesgo_jun.csv')

In [5]:
base_riesgo_jun = pd.read_csv('base_riesgo_jun.csv',sep=';')

In [62]:
base_riesgo_jun

,Unnamed: 0,PERIODO,SUBSCRIBER_ID,MSISDN,RIESGO,POSIBLE_OPERADOR,CON_DESCUENTO,GRUPO,ANTIGUEDAD,OFERTA,PERIODO_PROCESO_CODIGO,SUBSCRIBER_ID.1,MSISDN.1,ESTADO,MOVIMIENTO_TIPO_NOMBRE,PLAN_COMERCIAL_DK,CONSULTOR_DK,OPERADOR_PORTA_ORIGEN,OPERADOR_PORTA_DESTINO
0,0,20230905,192557534,3133647239,ALTO,CLARO,NO,GESTION,23.0,Dcto Retencion 20% X 5 meses,202309,192557534,3133647239,Active,PERMANECE,1769978,5771667,NaN,NaN
1,1,20230813,192559919,3204154565,ALTO,TIGO,NO,CONTROL,18.0,Dcto Retencion 10% X 6 meses,202309,192559919,3204154565,Active,PERMANECE,1125012,5806527,NaN,NaN
2,2,20230830,192560594,3102648746,ALTO,NA,NO,CONTROL,8.0,Dcto Retencion 10% X 6 meses,202309,192560594,3102648746,Active,PERMANECE,137539,6531904,NaN,NaN
3,3,20230810,192563174,3133026117,ALTO,MOVISTAR,NO,GESTION,16.0,Dcto Retencion 25% X 4 meses,202309,192563174,3133026117,Active,PERMANECE,1320273,5823576,AVANTEL,Wom
4,4,20230716,192572202,3502124984,ALTO,NA,NO,GESTION,11.0,Dcto Retencion 10% X 6 meses,202309,192572202,3502124984,Active,PERMANECE,1089095,6101944,AVANTEL,Wom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262826,263020,20230911,2488889560,3177947463,ALTO,CLARO,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,202309,2488889560,3177947463,Active,PERMANECE,1125007,6999998,TELEFONICA,Wom
262827,263021,20230911,2488925931,3016143255,ALTO,CLARO,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,202309,2488925931,3016143255,Active,PERMANECE,1769978,6972764,TELEFONICA,Wom
262828,263022,20230911,2488940086,3118044313,ALTO,MOVISTAR,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,202309,2488940086,3118044313,Active,PERMANECE,1125011,6854087,COMCEL,Wom
262829,263023,20230911,2489617765,3185142870,ALTO,MOVISTAR,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,202309,2489617765,3185142870,Active,PERMANECE,1125007,6905347,TELEFONICA,Wom


In [4]:
base_riesgo_jun.groupby(['ESTADO']).count()

,PERIODO,SUBSCRIBER_ID,MSISDN,RIESGO,POSIBLE_OPERADOR,CON_DESCUENTO,GRUPO,ANTIGUEDAD,OFERTA,PERIODO_PROCESO_CODIGO,SUBSCRIBER_ID,MSISDN,MOVIMIENTO_TIPO_NOMBRE,PLAN_COMERCIAL_DK,CONSULTOR_DK,OPERADOR_PORTA_ORIGEN,OPERADOR_PORTA_DESTINO
ESTADO,,,,,,,,,,,,,,,,,
Active,190577,190577,190577,190577,190577,190577,190577,187204,187204,190577,190577,190577,190577,190577,190577,159491,159487
One-Way Block,14493,14493,14493,14493,14493,14493,14493,14188,14188,14493,14493,14493,14493,14493,14493,10724,10724
Termination,56191,56191,56191,56191,56191,56191,56191,53490,53490,56191,56191,56191,56191,56191,56191,45300,50463
Two-Way Block,642,642,642,642,642,642,642,563,563,642,642,642,642,642,642,428,428


In [6]:
df = pd.read_excel('descuentos.xlsx')

In [49]:
df

,NUMERO,FECHA,RESPUESTA,PLAN_DESCUENTO
0,3008415203,2023-08-23,SI,Dcto Retencion 25% X 4 meses
1,3175517705,2023-08-23,SI,Dcto Retencion 25% X 4 meses
2,3028450643,2023-08-23,SI,Dcto Retencion 25% X 4 meses
3,3118091530,2023-08-23,SI,Dcto Retencion 25% X 4 meses
4,3507411460,2023-08-23,SI,Dcto Retencion 25% X 4 meses
...,...,...,...,...
247205,3197859716,2023-08-27,SIN_INTERACCION,NaN
247206,3045879146,2023-08-27,SIN_INTERACCION,NaN
247207,3212275577,2023-08-27,SIN_INTERACCION,NaN
247208,3186030218,2023-08-27,SIN_INTERACCION,NaN


In [3]:
df.describe()

,NUMERO
count,2.472100e+05
mean,3.131056e+09
std,9.580463e+07
min,3.001000e+09
25%,3.028486e+09
50%,3.132863e+09
75%,3.197481e+09
max,3.508999e+09


In [38]:
df_sin_duplicados1['MSISDN'] = df_sin_duplicados1['MSISDN'].astype('int64')

In [37]:
df_sin_duplicados1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262831 entries, 0 to 263024
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   PERIODO                 262831 non-null  int64  
 1   SUBSCRIBER_ID           262831 non-null  int64  
 2   MSISDN                  262831 non-null  object 
 3   RIESGO                  262831 non-null  object 
 4   POSIBLE_OPERADOR        262831 non-null  object 
 5   CON_DESCUENTO           262831 non-null  object 
 6   GRUPO                   262831 non-null  object 
 7   ANTIGUEDAD              256381 non-null  float64
 8   OFERTA                  256381 non-null  object 
 9   PERIODO_PROCESO_CODIGO  262831 non-null  int64  
 10  SUBSCRIBER_ID           262831 non-null  int64  
 11  MSISDN                  262831 non-null  object 
 12  ESTADO                  262831 non-null  object 
 13  MOVIMIENTO_TIPO_NOMBRE  262831 non-null  object 
 14  PLAN_COMERCIAL_DK   

In [40]:
df_sin_duplicados1 = df_sin_duplicados1.drop_duplicates(subset=['MSISDN'])

In [41]:
df_sin_duplicados1

,PERIODO,SUBSCRIBER_ID,MSISDN,RIESGO,POSIBLE_OPERADOR,CON_DESCUENTO,GRUPO,ANTIGUEDAD,OFERTA,PERIODO_PROCESO_CODIGO,SUBSCRIBER_ID,MSISDN,ESTADO,MOVIMIENTO_TIPO_NOMBRE,PLAN_COMERCIAL_DK,CONSULTOR_DK,OPERADOR_PORTA_ORIGEN,OPERADOR_PORTA_DESTINO
0,20230905,192557534,3133647239,ALTO,CLARO,NO,GESTION,23.0,Dcto Retencion 20% X 5 meses,202309,192557534,3133647239,Active,PERMANECE,1769978,5771667,None,None
1,20230813,192559919,3204154565,ALTO,TIGO,NO,CONTROL,18.0,Dcto Retencion 10% X 6 meses,202309,192559919,3204154565,Active,PERMANECE,1125012,5806527,None,None
2,20230830,192560594,3102648746,ALTO,NA,NO,CONTROL,8.0,Dcto Retencion 10% X 6 meses,202309,192560594,3102648746,Active,PERMANECE,137539,6531904,None,None
3,20230810,192563174,3133026117,ALTO,MOVISTAR,NO,GESTION,16.0,Dcto Retencion 25% X 4 meses,202309,192563174,3133026117,Active,PERMANECE,1320273,5823576,AVANTEL,Wom
4,20230716,192572202,3502124984,ALTO,NA,NO,GESTION,11.0,Dcto Retencion 10% X 6 meses,202309,192572202,3502124984,Active,PERMANECE,1089095,6101944,AVANTEL,Wom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263020,20230911,2488889560,3177947463,ALTO,CLARO,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,202309,2488889560,3177947463,Active,PERMANECE,1125007,6999998,TELEFONICA,Wom
263021,20230911,2488925931,3016143255,ALTO,CLARO,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,202309,2488925931,3016143255,Active,PERMANECE,1769978,6972764,TELEFONICA,Wom
263022,20230911,2488940086,3118044313,ALTO,MOVISTAR,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,202309,2488940086,3118044313,Active,PERMANECE,1125011,6854087,COMCEL,Wom
263023,20230911,2489617765,3185142870,ALTO,MOVISTAR,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,202309,2489617765,3185142870,Active,PERMANECE,1125007,6905347,TELEFONICA,Wom


In [24]:
base_riesgo_jun = base_riesgo_jun.drop_duplicates(subset=['MSISDN'])

In [30]:
df_sin_duplicados1.columns = df_sin_duplicados1.columns.str.upper()
df_sin_duplicados.columns = df_sin_duplicados.columns.str.upper()

In [7]:
df_insumos=base_riesgo_jun.merge(df,left_on='MSISDN',right_on='NUMERO',how='left')

In [65]:
df_insumos

,Unnamed: 0,PERIODO,SUBSCRIBER_ID,MSISDN,RIESGO,POSIBLE_OPERADOR,CON_DESCUENTO,GRUPO,ANTIGUEDAD,OFERTA,...,ESTADO,MOVIMIENTO_TIPO_NOMBRE,PLAN_COMERCIAL_DK,CONSULTOR_DK,OPERADOR_PORTA_ORIGEN,OPERADOR_PORTA_DESTINO,NUMERO,FECHA,RESPUESTA,PLAN_DESCUENTO
0,0,20230905,192557534,3133647239,ALTO,CLARO,NO,GESTION,23.0,Dcto Retencion 20% X 5 meses,...,Active,PERMANECE,1769978,5771667,NaN,NaN,NaN,NaT,NaN,NaN
1,1,20230813,192559919,3204154565,ALTO,TIGO,NO,CONTROL,18.0,Dcto Retencion 10% X 6 meses,...,Active,PERMANECE,1125012,5806527,NaN,NaN,NaN,NaT,NaN,NaN
2,2,20230830,192560594,3102648746,ALTO,NA,NO,CONTROL,8.0,Dcto Retencion 10% X 6 meses,...,Active,PERMANECE,137539,6531904,NaN,NaN,NaN,NaT,NaN,NaN
3,3,20230810,192563174,3133026117,ALTO,MOVISTAR,NO,GESTION,16.0,Dcto Retencion 25% X 4 meses,...,Active,PERMANECE,1320273,5823576,AVANTEL,Wom,NaN,NaT,NaN,NaN
4,4,20230716,192572202,3502124984,ALTO,NA,NO,GESTION,11.0,Dcto Retencion 10% X 6 meses,...,Active,PERMANECE,1089095,6101944,AVANTEL,Wom,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262831,263020,20230911,2488889560,3177947463,ALTO,CLARO,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,...,Active,PERMANECE,1125007,6999998,TELEFONICA,Wom,NaN,NaT,NaN,NaN
262832,263021,20230911,2488925931,3016143255,ALTO,CLARO,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,...,Active,PERMANECE,1769978,6972764,TELEFONICA,Wom,NaN,NaT,NaN,NaN
262833,263022,20230911,2488940086,3118044313,ALTO,MOVISTAR,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,...,Active,PERMANECE,1125011,6854087,COMCEL,Wom,NaN,NaT,NaN,NaN
262834,263023,20230911,2489617765,3185142870,ALTO,MOVISTAR,NO,GESTION,1.0,Dcto Retencion 20% X 5 meses,...,Active,PERMANECE,1125007,6905347,TELEFONICA,Wom,NaN,NaT,NaN,NaN


In [66]:
df_insumos.to_csv('base_riesgo_jun_completa2.csv')